This is the simple implementation of convolution LSTM network and prediction. I have used https://www.kaggle.com/unkownhihi/dfdc-lrcn-inference method to extract face and feed to neaural network.

In [ ]:
from keras import Input

from pathlib import Path
from keras.callbacks import Callback
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D, ConvLSTM2D, Reshape, BatchNormalization, Activation
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop

from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from collections import deque
import sys
import logging
import keras
from keras.applications import Xception, ResNet50, InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

from keras.models import load_model

from keras.callbacks import Callback, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder

from keras.utils import to_categorical

from sklearn.utils import class_weight, shuffle

import tensorflow as tf
import cv2
import glob
from tqdm.notebook import tqdm
import pandas as pd

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit



In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile('../input/conv-lstm-deepfake-weights/frozen_inference_graph_face.pb', 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(graph=detection_graph, config=config)
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
boxes_tensor = detection_graph.get_tensor_by_name('detection_boxes:0')
scores_tensor = detection_graph.get_tensor_by_name('detection_scores:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import keras
from scipy import io


import tensorflow as tf
import cv2
import glob
from tqdm.notebook import tqdm

import os

def get_img(images):
    global boxes,scores,num_detections
    im_heights,im_widths=[],[]
    imgs=[]
    for image in images:
        (im_height,im_width)=image.shape[:-1]
        imgs.append(image)
        im_heights.append(im_height)
        im_widths.append(im_widths)
    imgs=np.array(imgs)
    (boxes, scores_) = sess.run(
        [boxes_tensor, scores_tensor],
        feed_dict={image_tensor: imgs})
    finals=[]
    for x in range(boxes.shape[0]):
        scores=scores_[x]
        max_=np.where(scores==scores.max())[0][0]
        box=boxes[x][max_]
        ymin, xmin, ymax, xmax = box
        (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                      ymin * im_height, ymax * im_height)
        left, right, top, bottom = int(left), int(right), int(top), int(bottom)
        image=imgs[x]
        finals.append(cv2.cvtColor(cv2.resize(image[max([0,top-40]):bottom+80,max([0,left-40]):right+80],(128,128)),cv2.COLOR_BGR2RGB))
    finals=np.array(finals)
    return finals
def detect_video(video):
    frame_count=10
    capture = cv2.VideoCapture('../input/deepfake-detection-challenge/test_videos/'+video)
    v_len = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idxs = np.linspace(0,v_len,frame_count, endpoint=False, dtype=np.int)
    imgs=[]
    i=0
    for frame_idx in range(int(v_len)):
        ret = capture.grab()
        if not ret: 
            print("Error grabbing frame %d from movie %s" % (frame_idx, video))
        if frame_idx >= frame_idxs[i]:
            if frame_idx-frame_idxs[i]>20:
                return None
            ret, frame = capture.retrieve()
            if not ret or frame is None:
                print("Error retrieving frame %d from movie %s" % (frame_idx, video))
            else:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                imgs.append(frame)
            i += 1
            if i >= len(frame_idxs):
                break
    imgs=get_img(imgs)
    if len(imgs)<10:
        return None
    return np.stack(imgs)

class VideoSequenceGenerator(keras.utils.Sequence):

    def __init__(self, data_path,batch_size=64, shuffle = False):
        'Initialization'
        self.train_set = data_path 
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(self.train_set.shape[0])
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.train_set.shape[0]//self.batch_size

    def on_epoch_end(self):
        # update indexes after each epoch
        if self.shuffle == True:
            np.random.shuffle(self.indices)

		
    def __getitem__(self, index):
        #'Generate one batch of data'  
        X=np.zeros((self.batch_size,10,128,128,3))


        indexes = self.indices[index*self.batch_size:(index+1)*self.batch_size]

        for i in range (self.batch_size):
            data= detect_video(self.train_set.iloc[indexes[i]])


            # data=io.loadmat(self.train_set.path[indexes[i]])
            batch_x = data

        
            
        
            X[i,:,:]= batch_x
          
   

        return X




In [ ]:

seq = Sequential()
seq.add(Conv3D(filters=32, kernel_size=(2,2,2),input_shape=(10, 128, 128, 3),
               activation='relu',
               padding='same', data_format='channels_last'))
seq.add(BatchNormalization())
seq.add(ConvLSTM2D(filters=16, kernel_size=(2, 2),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())
seq.add(Dropout(0.3))

seq.add(MaxPooling3D(pool_size=(2,2,2)))

seq.add(ConvLSTM2D(filters=16, kernel_size=(2, 2),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())
seq.add(Dropout(0.3))



seq.add(MaxPooling3D(pool_size=(2,2,2)))
seq.add(Flatten())

seq.add(BatchNormalization())

seq.add(Dense(32,activation='elu'))

seq.add(Dense(1,activation='sigmoid'))

In [ ]:
model=seq
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

In [ ]:
# os.mkdir('./videos/')
# for x in tqdm(glob.glob('../input/deepfake-detection-challenge/test_videos/*.mp4')):
#     try:
#         filename=x.replace('../input/deepfake-detection-challenge/test_videos/','').replace('.mp4','.jpg')
#         a=detect_video(x)
#         if a is None:
#             continue
#         cv2.imwrite('./videos/'+filename,a)
#     except Exception as err:
#         print(err)


In [ ]:

model.load_weights('../input/conv-lstm-deepfake-weights/model-12-0.31.h5')

In [ ]:
sample_submission = pd.read_csv("../input/deepfake-detection-challenge/sample_submission.csv")

submission=pd.DataFrame()
submission['filename']=os.listdir(('../input/deepfake-detection-challenge/test_videos/'))
submission['label']=0.5
filenames=[]
batch=[]
batch1=[]
preds=[]

In [ ]:
x_name=submission['filename']

In [ ]:
pre=VideoSequenceGenerator(x_name,8)

In [ ]:
preds = model.predict_generator(pre, verbose = 1)

In [ ]:
filenames=submission['filename']

In [ ]:
new_preds=[]
for x in preds:
    new_preds.append(x[0])
print(sum(new_preds)/len(new_preds))

In [ ]:
for x,y in zip(new_preds,filenames):
    submission.loc[submission['filename']==y,'label']=min([max([0.0,x]),1.0])

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.hist(submission['label'])

In [ ]:
np.array(submission['label']).mean()

In [ ]:
submission.to_csv('submission.csv', index=False)